# Numerical methods for finding roots of polynomials

Let us consider the polynomial $$ p(x) = 2x^6 +  25x^5 -  4x^4 + 13x^3 + 172x^2 - 7x - 24. $$

In [1]:
using Printf

In [2]:
using Polynomials

# p(x) = 2x^6 +  25x^5 -  4x^4 + 13x^3 + 172x^2 - 7x - 24
a = reverse(  Array{Float64,1}([ 2, 25, -4, 13, 172, -7, -24 ]) )
p = Poly(a)

Poly(-24.0 - 7.0*x + 172.0*x^2 + 13.0*x^3 - 4.0*x^4 + 25.0*x^5 + 2.0*x^6)

In [3]:
using Plotly
plot( x -> p(x), -2, 2 )

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/a3031b2c48b2879caffc1e55af1c24b5fb8a1364-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/a3031b2c48b2879caffc1e55af1c24b5fb8a1364-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

data: [
  "scatter with fields name, type, x, and y"
]

layout: "layout with field margin"

## Computing derivatives with Horner's method

In [4]:
# Horner algorithm computing p(z) with given coefficients of p
function Horner0( a, z )
    n = length(a)
    p = a[n]
    for k = n-1:-1:1            
        p = a[k] + z*p
    end
    return p
end

# Horner algorithms computing p(z) and p'(z) with given coefficients of p
function Horner1( a, z )
    n  = length(a)
    p  = a[n]
    ∂p = 0
    for k = n-1:-1:1            
        ∂p  = p    + z*∂p
        p   = a[k] + z*p
    end
    return p, ∂p
end

# Horner algorithms computing p(z),p'(z),p''(z) with given coefficients of p
function Horner2( a, z )
    n   = length(a)
    p   = a[n-1] + z*a[n]
    ∂p  = a[n]
    ∂∂p = zero( z )
    for k = n-2:-1:1            
        ∂∂p = ∂p   + z*∂∂p
        ∂p  = p    + z*∂p
        p   = a[k] + z*p
    end
    return p, ∂p, ∂∂p*2
end


# Test of Horner2
function test_Horner2( coeffs, z )
    p   = Poly(coeffs)
    dp  = polyder(p)
    ddp = polyder(dp)
    a,b,c = Horner2( coeffs, z )
    @assert( abs(a - p(z)) < 1e-10 )
    @assert( abs(b- dp(z)) < 1e-10 )
    @assert( abs(c-ddp(z)) < 1e-10 )
end

test_Horner2( a,  0.78  )
test_Horner2( a,  0.178 )
test_Horner2( a,  5.78 )
test_Horner2( a, -6.78 )

In [5]:
# Remark: the following test fails
test_Horner2( a, -12.78 )

AssertionError: AssertionError: abs(b - dp(z)) < 1.0e-10

## Newton's method

In [6]:
# Newton's method
# Input :
#   coeffs - coefficients of polynomial
#   x0     - initial approximation
#
# Optional parameters:
#   *) stopping criteria
#        ɛs   - relative error of two consecutive approximations
#        imax - max number of iterations
#   *) verbose mode
#        print = true | false
#
# Output
#   xr   - approximation of the root
#   ϵ    - absolute error estimation
#   iter - number of iterations performed
#
function newton( coeffs, x0; ɛs=1.0e-6, imax=20, print=false )
    iter = 0; 
    xr = x0; 
    ɛa = one(x0)
    xrs = [xr]
    if print; @printf("Iter %2d : xr=%10.7f  ɛa=%.3e\n" , iter, xr, ɛa); end;
    while true
        xrold = xr
        iter  = iter+1
        f,df = Horner1( coeffs, xrold )
        xr    = xrold - f/df
        push!(xrs,xr)
        if xr!=0.0
            ɛa = abs((xr-xrold)/xr)
        end;
        if print; @printf("Iter %2d : xr=%10.7f  ɛa=%.3e\n" , iter, xr, ɛa); end;
        if ɛa<ɛs || iter>=imax
            break;
        end
    end
    return xr,xrs,iter,ɛa;
end;

In [7]:
newton( a, -2.0 , print=true)

Iter  0 : xr=-2.0000000  ɛa=1.000e+00
Iter  1 : xr=-1.8655602  ɛa=7.206e-02
Iter  2 : xr=-1.8346276  ɛa=1.686e-02
Iter  3 : xr=-1.8330839  ɛa=8.421e-04
Iter  4 : xr=-1.8330802  ɛa=2.037e-06
Iter  5 : xr=-1.8330802  ɛa=1.190e-11


(-1.833080209420786, [-2.0, -1.86556, -1.83463, -1.83308, -1.83308, -1.83308], 5, 1.1904728235717812e-11)

In [8]:
newton( a, -1.0 )

(-0.360075794873698, [-1.0, -0.371585, -0.360225, -0.360076, -0.360076], 4, 7.240078984326531e-8)

In [9]:
newton( a, 0.5 )

(0.38745680836108753, [0.5, 0.402245, 0.387769, 0.387457, 0.387457], 4, 3.7325544295269393e-7)

In [10]:
# All roots (using Polynomials package)
roots(p)

6-element Array{Complex{Float64},1}:
  -12.656084636134612 + 0.0im               
   0.9808919160340197 + 1.6569153010117608im
   0.9808919160340197 - 1.6569153010117608im
  -1.8330802094207865 + 0.0im               
  0.38745680836105684 + 0.0im               
 -0.36007579487369734 + 0.0im               

In [11]:
newton( a, 1.0 + 1.0im ) # --> converges to real root 0.387457

(0.3874568083610565 - 6.6763484318441225e-19im, Complex{Float64}[1.0+1.0im, 0.263696+0.973095im, 0.271345+0.342313im, 0.233261+0.0360993im, 0.437278-0.0357529im, 0.389393-0.00455093im, 0.387432-2.62627e-5im, 0.387457+1.92332e-9im, 0.387457-6.67635e-19im], 8, 4.9731557125783815e-9)

In [12]:
newton( a, 1.0 + 1.5im ) # --> converges to complex root 0.980892+1.65692im

(0.9808919160340199 + 1.6569153010117617im, Complex{Float64}[1.0+1.5im, 0.936748+1.68345im, 0.97838+1.65355im, 0.980911+1.65694im, 0.980892+1.65692im, 0.980892+1.65692im], 5, 6.657124243160878e-10)

Hence the problem is to find ***good initial approximation***.

# Bairstow's method
TODO: translate polish -> english

## Wprowadzenie
Rozpatrzmy wielomian
$$ p(z) = a_n z^n + a_{n-1} z^{n-1} + \ldots + a_0. $$

**Twierdzenie 1.**
Dzielenie wielomianu $p(z)$ przez czynnik kwaratowy $z^2 - uz - v$ daje iloraz i resztę, równe odpowiednio
$$ \begin{align*}
q(z) &:= b_n z^{n-2} + b_{n-1} z^{n-3} + \ldots + b_3z + b_2,\\
r(z) &:= b_1(z-u) + b_0,
\end{align*}$$
których współczynniki można obliczać rekurencyjnie według wzorów
$$ b_{n+1}=b_{n+2}=0, \qquad b_k = a_k + ub_{k+1} + vb_{k+2} \quad (n\geq k\geq 0). $$

Dowód [Tw. 3.5.13, s. 108]{KincaidCheney}.


Wprowadźmy oznaczenia
$$ c_k := \frac{\partial b_k}{\partial u}, \qquad d_k := \frac{\partial b_{k-1}}{\partial v}. $$
**Twierdzenie 2.**
Wielkości $c_k$ i $d_k$ spełniają ten sam związek rekurencyjny z takimi samymi wartościami początkowymi:
$$ \begin{align*} c_k &= b_{k+1} + uc_{k+1} + vc_{k+2} \qquad (c_{n+1}=c_n=0),\\
d_k &= b_{k+1} + ud_{k+1} + vd_{k+2} \qquad (d_{n+1}=d_n=0).\end{align*}$$

**Wniosek 3.**
Zachodzi równość $c_k=d_k$.

## Idea metody
Aby znaleźć pierwiastki wielomianu $p(z)$, szukamy takiego czynnika kwadratowego $z^2-uz-v$, dla którego reszta $r(z)$ jest zerem, tzn. $b_1=b_0=0$.
Ponieważ $$ b_1 = b_1(u,v), \qquad b_0=b_0(u,v),$$
więc szukamy takich wartości $u,v$, dla których spełnione są dwa równania
$$\begin{align*} b_1(u,v) = 0,\\ b_0(u,v)=0. \end{align*}$$
Są to równanie nieliniowe ze względu na $u,v$. Do znalezienia $u,v$ stosujemy zatem metodę Newtona.
W $n$ tym kroku metody poszukujemy więc takich poprawek $h_n^{(u)}$, $h_n^{(v)}$, dla których zachodzą następujące dwa równania **liniowe**
$$ \begin{align*}
b_0(u_n, v_n) + \frac{\partial b_0}{\partial u}(u_n, v_n) \cdot h_n^{(u)} + 
\frac{\partial b_0}{\partial v}(u_n, v_n) \cdot h_n^{(v)} = 0, \\
b_1(u_n, v_n) + \frac{\partial b_1}{\partial u}(u_n, v_n) \cdot h_n^{(u)} + 
\frac{\partial b_1}{\partial v}(u_n, v_n) \cdot h_n^{(v)} = 0.
\end{align*}$$

Wobec wniosku 3, równanie te można zapisać równoważnie w postaci:
$$ \begin{align*}
b_0(u_n, v_n) + c_0(u_n, v_n) \cdot h_n^{(u)} + 
c_1(u_n, v_n) \cdot h_n^{(v)} = 0, \\
b_1(u_n, v_n) + c_1(u_n, v_n) \cdot h_n^{(u)} + 
c_2(u_n, v_n) \cdot h_n^{(v)} = 0.
\end{align*}$$
Stąd
$$ \begin{align*}
h_n^{(u)} = (c_1 b_1 - c_2 b_0) / J,\\
h_n^{(v)} = (c_1 b_0 - c_0 b_1) / J,
\end{align*}\qquad J=c_0c_2-c_1^2. $$

In [13]:
# wektor współczynników a_0, a_1, ..., a_n
@show a = Array{Float64,1}( a )
n = length(a)-1;

a = Array{Float64, 1}(a) = [-24.0, -7.0, 172.0, 13.0, -4.0, 25.0, 2.0]


In [14]:
roots(Poly(a))

6-element Array{Complex{Float64},1}:
  -12.656084636134612 + 0.0im               
   0.9808919160340197 + 1.6569153010117608im
   0.9808919160340197 - 1.6569153010117608im
  -1.8330802094207865 + 0.0im               
  0.38745680836105684 + 0.0im               
 -0.36007579487369734 + 0.0im               

### Jeden krok metody Bairstowa

In [15]:
maxiter = 10        # maksymalna liczba iteracji w metodzie Newtona

b = zeros(Float64, size(a))
b[n+1] = a[n+1]     # b_n     = a_n
c = zeros(Float64, size(a))
c[n+1] = 0.0        # c_n     = 0
c[n]   = a[n+1]     # c_{n-1} = a_n

u,v = 0.0, 0.0
for j = 1:maxiter
    b[n] = a[n] + u*b[n+1]
    for k=n-2:-1:0
        b[k+1] = a[k+1] + u*b[k+2] + v*b[k+3]
        c[k+1] = b[k+2] + u*c[k+2] + v*c[k+3]
    end
    J = c[1]*c[3] - c[2]*c[2]
    u = u + (c[2]*b[2] - c[3]*b[1])/J
    v = v + (c[2]*b[1] - c[1]*b[2])/J
    @show j,u,v,b[1],b[2]
end

@show roots( Poly([-v,-u,1.0]) )

(j, u, v, b[1], b[2]) = (1, 0.030058972198820557, 0.14075821398483573, -24.0, -7.0)
(j, u, v, b[1], b[2]) = (2, 0.027381242843261912, 0.1395218955916894, 0.2358316843262216, 0.4862738375212836)
(j, u, v, b[1], b[2]) = (3, 0.027381013496904485, 0.13951381824955633, 0.00139329120004561, 0.00019992227261145246)
(j, u, v, b[1], b[2]) = (4, 0.027381013487359312, 0.13951381824983322, 2.3771207224854152e-11, 1.6389551937834312e-9)
(j, u, v, b[1], b[2]) = (5, 0.027381013487359312, 0.13951381824983322, 0.0, 0.0)
(j, u, v, b[1], b[2]) = (6, 0.027381013487359312, 0.13951381824983322, 0.0, 0.0)
(j, u, v, b[1], b[2]) = (7, 0.027381013487359312, 0.13951381824983322, 0.0, 0.0)
(j, u, v, b[1], b[2]) = (8, 0.027381013487359312, 0.13951381824983322, 0.0, 0.0)
(j, u, v, b[1], b[2]) = (9, 0.027381013487359312, 0.13951381824983322, 0.0, 0.0)
(j, u, v, b[1], b[2]) = (10, 0.027381013487359312, 0.13951381824983322, 0.0, 0.0)
roots(Poly([-v, -u, 1.0])) = [0.387457, -0.360076]


2-element Array{Float64,1}:
  0.38745680836105656
 -0.36007579487369723

Można zauważyć, że wektor ***b*** zawiera współczynniki ilorazu $q(z)$. Wystarczy zatem wykonać ponownie krok metody dla wektora b[3:end].

In [16]:
function Bairstow(a::Vector{Float64}; maxiter=15)    
    # maxiter = maksymalna liczba iteracji w metodzie Newtona
    
    function solve_quadratic_equation(a::Float64,b::Float64,c::Float64)
        Δ=b*b-4.0*a*c;
        x1,x2 = 0.0, 0.0;
        
        if (Δ>0.0)
            sΔ = sqrt(Δ)
            if (b>0.0)
                x1 = (-b-sΔ)/(2.0*a)
                x2 = c/x1
            else
                x2 = (-b+sΔ)/(2.0*a)
                x1 = c/x2
            end
        elseif (Δ<0.0)
            sΔ = sqrt(-Δ)
            if (b>0.0)
                x1 = (-b-sΔ*im)/(2.0*a)
                x2 = c/x1
            else
                x2 = (-b+sΔ*im)/(2.0*a)
                x1 = c/x2
            end
        else
            x1 = -b/(2.0*a);
            x2 = -b/(2.0*a);
        end
        return x1,x2;
    end      
    
    n = length(a)-1;
    α = zeros( Complex{Float64}, n ); _i = 1;
    while (n>1)
        b = zeros(Float64, size(a))
        b[n+1] = a[n+1]     # b_n     = a_n
        c = zeros(Float64, size(a))
        c[n+1] = 0.0        # c_n     = 0
        c[n]   = a[n+1]     # c_{n-1} = a_n

        u,v = 0.0, 0.0
        for j = 1:maxiter
            b[n] = a[n] + u*b[n+1]
            for k=n-2:-1:0
                b[k+1] = a[k+1] + u*b[k+2] + v*b[k+3]
                c[k+1] = b[k+2] + u*c[k+2] + v*c[k+3]
            end
            J = c[1]*c[3] - c[2]*c[2]
            u = u + (c[2]*b[2] - c[3]*b[1])/J
            v = v + (c[2]*b[1] - c[1]*b[2])/J
            j,u,v,b[1],b[2]
        end
        x1,x2 = solve_quadratic_equation(1.0,-u,-v)
        α[_i] = x1; α[_i+1] = x2; _i = _i+2;
        a = b[3:end]
        n = n-2;
    end
    if (n==1)
        α[_i] = -a[1]/a[2]
    end
    return α
end

Bairstow (generic function with 1 method)

In [17]:
Bairstow(a)

6-element Array{Complex{Float64},1}:
 -0.36007579487369723 + 0.0im               
  0.38745680836105656 + 0.0im               
  -12.656084636134612 + 0.0im               
  -1.8330802094207865 + 0.0im               
     0.98089191603402 - 1.6569153010117612im
   0.9808919160340199 + 1.656915301011761im 

In [18]:
roots(Poly(a))

6-element Array{Complex{Float64},1}:
  -12.656084636134612 + 0.0im               
   0.9808919160340197 + 1.6569153010117608im
   0.9808919160340197 - 1.6569153010117608im
  -1.8330802094207865 + 0.0im               
  0.38745680836105684 + 0.0im               
 -0.36007579487369734 + 0.0im               

In [19]:
coeffs = Polynomials.coeffs

coeffs (generic function with 1 method)

In [20]:
using LinearAlgebra
### Przykład 2

# Ustawiamy pierwiastki wielomianu
rts = [ 3.0, 4.0, 10.0, 18.0, 24.0, 30.0, -5.0, -6.0, 5.0, 5.0, 5.0, 5.0]
a = coeffs( poly( rts ) )

rb = Bairstow(a; maxiter = 10000)
LinearAlgebra.norm( sort( real(rb) ) - sort(rts) )

0.016656600177063538

In [21]:
### Przykład 3

# Ustawiamy pierwiastki wielomianu
rts = rand(10)
a = coeffs( poly( rts ) )

rB = Bairstow(a; maxiter = 50)

LinearAlgebra.norm( sort( real(rB) ) - sort( rts ) )

3.288790718572662e-8

In [22]:
### Przykład 4

# Losujemy współczynniki wielomianu

@show a = rand(15)
@show rB = Bairstow(a; maxiter=50)
@show rP = roots( Poly(a) )

sort( abs.(rB) ) - sort( abs.(rP) )

a = rand(15) = [0.838731, 0.798022, 0.737902, 0.966599, 0.71755, 0.0229473, 0.289302, 0.58291, 0.212543, 0.791221, 0.583948, 0.915591, 0.999375, 0.951768, 0.47846]
rB = Bairstow(a; maxiter=50) = Complex{Float64}[-0.748271-0.541262im, -0.748271+0.541262im, 0.222994-0.88822im, 0.222994+0.88822im, -2.53909+0.0im, 2.45173+0.0im, 2.18328-1.6865im, 2.18328+1.6865im, -2.32472-1.54479im, -2.32472+1.54479im, 0.714469-2.672im, 0.714469+2.672im, -1.01494-2.59272im, -1.01494+2.59272im]
rP = roots(Poly(a)) = Complex{Float64}[-1.43439+0.0im, -0.729375+1.00691im, -0.729375-1.00691im, -0.962187+0.0im, -0.748271+0.541262im, -0.748271-0.541262im, 0.922356+0.348622im, 0.922356-0.348622im, 0.625939+0.800888im, 0.625939-0.800888im, -0.0899703+1.06555im, -0.0899703-1.06555im, 0.222994+0.88822im, 0.222994-0.88822im]


14-element Array{Float64,1}:
 -2.220446049250313e-16 
 -2.220446049250313e-16 
  2.220446049250313e-16 
  3.3306690738754696e-16
  1.4895453739555495    
  1.5530509251634306    
  1.7727570382680957    
  1.742323436096402     
  1.7493983412776712    
  1.6965354482285344    
  1.7149579609335504    
  1.5409748195785273    
  1.5478662151330715    
  1.3567962972810346    

In [23]:
using Random
###
### Niestety metoda nie działa dla większych wartości n
Random.seed!(1)

@show a = rand(20)
rB = Bairstow(a; maxiter=50)

a = rand(20) = [0.236033, 0.346517, 0.312707, 0.00790928, 0.488613, 0.210968, 0.951916, 0.999905, 0.251662, 0.986666, 0.555751, 0.437108, 0.424718, 0.773223, 0.28119, 0.209472, 0.251379, 0.0203749, 0.287702, 0.859512]


19-element Array{Complex{Float64},1}:
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im
 NaN + 0.0im

## Laguerre's method

In [24]:
# Laguerre's method
# Input :
#   coeffs - coefficients of polynomial
#   x0     - initial approximation
#
# Optional parameters:
#   *) stopping criteria
#        ɛs   - relative error of two consecutive approximations
#        imax - max number of iterations
#   *) verbose mode
#        print = true | false
#
# Output
#   xr   - approximation of the root
#   ϵ    - absolute error estimation
#   iter - number of iterations performed
#
function Laguerre( coeffs, x0; ɛs=1.0e-6, imax=20, print=false )
    iter = 0;
    n = length(coeffs)-1;
    xr = x0; 
    ɛa = one(x0)
    xrs = [xr]
    if print; @printf("Iter %2d : xr=%10.7f  ɛa=%.3e\n" , iter, xr, ɛa); end;
    while true
        xrold = xr
        iter  = iter+1
        f,df,ddf = Horner2( coeffs, xrold )
        H = (n-1)*((n-1)*df^2 - n*f*ddf)
        xr1    = xrold - n*f/(df + sqrt(H))
        xr2    = xrold - n*f/(df - sqrt(H))
        if ( abs(xr1-xrold) < abs(xr2-xrold) )
            xr = xr1
        else
            xr = xr2
        end
        push!(xrs,xr)
        if xr!=zero(x0)
            ɛa = abs((xr-xrold)/xr)
        end;
        if print; @printf("Iter %2d : xr=%10.7f  ɛa=%.3e\n" , iter, xr, ɛa); end;
        if ɛa<ɛs || iter>=imax
            break;
        end
    end
    return xr,xrs,iter,ɛa;
end;

In [25]:
newton( a, -1.0, imax=50, εs=1e-10 )

(-0.9214602452910814, [-1.0, -0.954718, -0.928949, -0.921903, -0.921462, -0.92146, -0.92146], 6, 2.3838715547716864e-11)

In [26]:
Laguerre( a, -1.0, imax=100, εs=1e-10, print=true )

Iter  0 : xr=-1.0000000  ɛa=1.000e+00
Iter  1 : xr=-0.9163213  ɛa=9.132e-02
Iter  2 : xr=-0.9214613  ɛa=5.578e-03
Iter  3 : xr=-0.9214602  ɛa=1.197e-06
Iter  4 : xr=-0.9214602  ɛa=0.000e+00


(-0.9214602452910814, [-1.0, -0.916321, -0.921461, -0.92146, -0.92146], 4, 0.0)

In [27]:
setprecision(5000)
big_a = Array{BigFloat,1}(a);

In [28]:
newton( a, BigFloat(-1.0), imax=50, εs=BigFloat(1e-200), print=true );

Iter  0 : xr=-1.0000000  ɛa=1.000e+00
Iter  1 : xr=-0.9547180  ɛa=4.743e-02
Iter  2 : xr=-0.9289488  ɛa=2.774e-02
Iter  3 : xr=-0.9219034  ɛa=7.642e-03
Iter  4 : xr=-0.9214619  ɛa=4.792e-04
Iter  5 : xr=-0.9214602  ɛa=1.765e-06
Iter  6 : xr=-0.9214602  ɛa=2.384e-11
Iter  7 : xr=-0.9214602  ɛa=4.349e-21
Iter  8 : xr=-0.9214602  ɛa=1.447e-40
Iter  9 : xr=-0.9214602  ɛa=1.603e-79
Iter 10 : xr=-0.9214602  ɛa=1.966e-157
Iter 11 : xr=-0.9214602  ɛa=2.958e-313


In [29]:
Laguerre( a, BigFloat(-1.0), imax=50, εs=BigFloat(1e-200), print=true );

Iter  0 : xr=-1.0000000  ɛa=1.000e+00
Iter  1 : xr=-0.9163213  ɛa=9.132e-02
Iter  2 : xr=-0.9214613  ɛa=5.578e-03
Iter  3 : xr=-0.9214602  ɛa=1.197e-06
Iter  4 : xr=-0.9214602  ɛa=1.200e-17
Iter  5 : xr=-0.9214602  ɛa=1.207e-50
Iter  6 : xr=-0.9214602  ɛa=1.232e-149
Iter  7 : xr=-0.9214602  ɛa=1.307e-446


Concluding, one can see the **cubic** convergence of Laguerre's method.